In [8]:
import re
import pandas as pd
from datetime import datetime, timedelta
import locale


# ====================================================
# Hühnerprotokoll Chat aus Signal exportieren
# ====================================================
# Installieren von https://github.com/carderne/signal-export
# export mit folgendem command:
# sigexport --paginate=0 --chats="Hühner Protokoll 🐓📖" ~/signal-chats
# chat.md Datei in gleiches Verzeichnis kopieren wie dieses Skript


# ====================================================
# Locale setzen für deutsche Wochentage
# ====================================================
# locale.LC_TIME beeinflusst die Ausgabe von Datumsformaten wie strftime("%A")
# "de_DE.UTF-8" ist unter Linux/macOS üblich.
# Falls das System diese Locale nicht hat, fällt es auf den Default zurück.
try:
    locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")
except locale.Error:
    pass


# ====================================================
# Vorverarbeitung des Chat-Exports
# ====================================================
def preprocess_chat(raw_text: str, author: str = "Axel") -> str:
    """
    Entfernt Zitate, Emoji-Reaktionen und Bilder aus dem Chat-Export
    und ersetzt 'Me' durch den angegebenen Autor.
    """
    cleaned = raw_text

    # 1) Zitate: beginnen mit '>' am Zeilenanfang
    # Regex: ^>.*$   → ganze Zeile, die mit '>' beginnt, wird entfernt
    cleaned = re.sub(r"^>.*$", "", cleaned, flags=re.M)

    # 2) Emoji-Reaktionen:
    # Beispiel: "(- Person: 👍 -)"
    # Regex: \( - ... - \) entfernt alles zwischen (- und -)
    cleaned = re.sub(r"\(-\s.*?:.*?-\)", "", cleaned)

    # 3) Bilder/Media-Links:
    # Beispiel: [2025-08-09T08-56-26.363_00_None.jpeg](./media/...)
    # Regex: !?\[[^\]]*?\.(jpg|jpeg|png|gif|heic|webp)\]\([^\)]*\)
    # entfernt Markdown-Links zu Bildern
    cleaned = re.sub(
        r"!?\[[^\]]*?\.(?:jpg|jpeg|png|gif|heic|webp|mp4)\]\([^\)]*\)",
        "",
        cleaned,
        flags=re.I,
    )

    # 4) "Me" durch Autor ersetzen (nur ganzes Wort)
    cleaned = re.sub(r"\bMe\b", author, cleaned)

    # Mehrfache Leerzeilen zusammenfassen
    cleaned = re.sub(r"\n{3,}", "\n\n", cleaned)
    return cleaned


# ====================================================
# Regex / Konstanten
# ====================================================

# Nachrichten-Erkennung:
# Format: [YYYY-MM-DD HH:MM:SS] Person: Nachricht
# Regex erklärt:
#   \[(.*?)\]   → Zeitstempel in eckigen Klammern
#   (.*?)       → Name (bis zum Doppelpunkt)
#   (.*?)       → Nachricht (greedy, bis zur nächsten doppelten Leerzeile oder Ende)
MSG_REGEX = re.compile(r"\[(.*?)\]\s(.*?):\s*(.*?)(?=\n\n\[|$)", re.S)

# Wochentage (Hinweise im Nachrichtentext, falls Nachricht verspätet geschrieben wurde)
WEEKDAY_HINTS = {
    "montag": 0, "mo ": 0, "mo-": 0,
    "dienstag": 1, "di ": 1, "di-": 1,
    "mittwoch": 2, "mi ": 2, "mi-": 2,
    "donnerstag": 3, "do ": 3, "do-": 3,
    "freitag": 4, "fr ": 4, "fr-": 4,
    "samstag": 5, "sa ": 5, "sa-": 5,
    "sonntag": 6, "so ": 6, "so-": 6,
}

# Zahlwörter, die wir zusätzlich zu Ziffern erkennen wollen
NUM_WORDS = {
    "ein": 1, "eine": 1, "eins": 1, "einen": 1, "einem": 1, "einer": 1,
    "zwei": 2,
    "drei": 3,
    "vier": 4,
    "fünf": 5, "fuenf": 5,
    "sechs": 6,
    "sieben": 7,
    "acht": 8,
    "neun": 9,
    "zehn": 10,
}

# Regex: Eier-Zahlen finden
# (\d+)\s*ei(?:er)?\b  → z. B. "1 Ei", "2 Eier"
RE_EGGS_NUMERIC = re.compile(r"(\d+)\s*ei(?:er)?\b", re.I)

# Regex: Zahlwörter + Eier finden (= gelegte EIer)
# \b(ein|zwei|...) ei(er)? → z. B. "zwei Eier"
RE_EGGS_WORDS = re.compile(
    r"\b(" + "|".join(map(re.escape, NUM_WORDS.keys())) + r")\s*ei(?:er)?\b",
    re.I
)

# Regex: Eier an Mitglieder weitergegeben
# z. B. "2 Eier an Fritz", "ein Ei für Lisa"
RE_MEMBER_GIVE = re.compile(
    r"((?:\d+)|(?:" + "|".join(map(re.escape, NUM_WORDS.keys())) + r"))\s*ei(?:er)?\s+(?:an|für)\s+\S+",
    re.I
)

# Regex: Eier mitgenommen (sollen ignoriert werden bei # Eier)
# Beispiel: "2 Eier mitgenommen", "ein Ei mitgenommen"
RE_MITGEN = re.compile(
    r"((?:\d+)|(?:" + "|".join(map(re.escape, NUM_WORDS.keys())) + r"))\s*ei(?:er)?(?:\s+\w{1,20}){0,3}\s*mitgenommen",
    re.I
)


# ====================================================
# Hilfsfunktionen
# ====================================================

def detect_service(text: str, ts: datetime) -> str:
    """
    Ermittelt, ob Dienst Morgens oder Abends war.
    - Explizit anhand Schlüsselwörtern im Text (z. B. 'Morgens', 'Abends').
    - Falls nicht angegeben: Uhrzeit des Zeitstempels nutzen.
    """
    t = text.lower()
    if "morg" in t or "vormittag" in t:
        return "Morgens"
    if "abend" in t or "abends" in t or "sa abend" in t or "so abend" in t:
        return "Abends"
    return "Morgens" if ts.hour < 12 else "Abends"


def parse_int_or_word(token: str) -> int:
    """Hilfsfunktion: wandelt Ziffern oder Zahlwörter in Integer um."""
    token = token.lower()
    if token.isdigit():
        return int(token)
    return NUM_WORDS.get(token, 0)


def extract_eggs_members_and_mask(text: str) -> tuple[int, str]:
    """
    Zählt Abgaben an Mitglieder (z. B. '2 Eier an Fritz').
    Entfernt diese Segmente aus dem Text, damit sie nicht doppelt als gelegte Eier gezählt werden.
    """
    lowered = text.lower()
    total_members = 0
    spans = []
    for m in RE_MEMBER_GIVE.finditer(lowered):
        num_token = m.group(1)
        total_members += parse_int_or_word(num_token)
        spans.append(m.span())
    if not spans:
        return total_members, lowered
    # Segmente entfernen (von hinten nach vorne, um Indizes stabil zu halten)
    masked = lowered
    for start, end in sorted(spans, reverse=True):
        masked = masked[:start] + " " + masked[end:]
    return total_members, masked


def mask_mitgenommen_segments(text: str) -> str:
    """
    Entfernt Segmente wie '2 Eier mitgenommen' aus dem Text,
    damit sie nicht als gelegte Eier gezählt werden.
    """
    lowered = text.lower()
    spans = []
    for m in RE_MITGEN.finditer(lowered):
        spans.append(m.span())
    if not spans:
        return lowered
    masked = lowered
    for start, end in sorted(spans, reverse=True):
        masked = masked[:start] + " " + masked[end:]
    return masked


def extract_eggs_laid(text_without_member_gives: str) -> int:
    """
    Zählt die gelegten Eier im Text (nach Entfernen von Mitglieder-Abgaben und 'mitgenommen').
    """
    t = text_without_member_gives.lower()
    total = 0
    # Ziffern
    for n in RE_EGGS_NUMERIC.findall(t):
        total += int(n)
    # Zahlwörter
    for word in RE_EGGS_WORDS.findall(t):
        total += NUM_WORDS.get(word.lower(), 0)
    return total


def adjust_date_by_weekday(text: str, timestamp: datetime) -> datetime.date:
    """
    Falls Nachricht verspätet gesendet wurde (z. B. 'Dienstag' im Text),
    korrigiere das Datum auf den ersten passenden Wochentag vor dem Zeitstempel.
    """
    t = text.lower()
    for hint, wd in WEEKDAY_HINTS.items():
        if hint in t:
            for delta in range(0, 7):
                candidate = (timestamp.date() - timedelta(days=delta))
                if candidate.weekday() == wd:
                    return candidate
    return timestamp.date()


# ====================================================
# Hauptlogik: Einlesen, Parsen, Gruppieren, Speichern
# ====================================================

FILE_PATH = "chat.md"
AUTHOR = "DeinName" #Hier den Namen eingeben, der Person, die den Signal chat exportiert hat damit "me" mit dem Namen ersetzt wird.
FILTER_YEAR = None #zB 2025; auf None setzen, wenn alle Jahre geparst werden sollen

# 1) Datei einlesen
with open(FILE_PATH, "r", encoding="utf-8") as f:
    raw = f.read()

# 2) Vorverarbeitung
raw = preprocess_chat(raw, author=AUTHOR)

# 3) Nachrichten parsen
matches = MSG_REGEX.findall(raw)
messages = []
for ts, person, text in matches:
    text = text.strip()
    if not text:
        continue
    try:
        timestamp = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        continue
    if FILTER_YEAR and timestamp.year != FILTER_YEAR:
        continue
    messages.append({"timestamp": timestamp, "person": person.strip(), "text": text})

# 4) Nachrichten zu Tages-Einträgen zusammenfassen
records = {}
for msg in messages:
    corrected_date = adjust_date_by_weekday(msg["text"], msg["timestamp"])
    service = detect_service(msg["text"], msg["timestamp"])
    key = (corrected_date, service)

    # Abgaben an Mitglieder zählen + aus Text entfernen
    members_count, masked_text = extract_eggs_members_and_mask(msg["text"])
    # "mitgenommen"-Segmente entfernen
    masked_text_no_mitgenomen = mask_mitgenommen_segments(masked_text)
    # gelegte Eier zählen
    laid_count = extract_eggs_laid(masked_text_no_mitgenomen)

    if key not in records:
        records[key] = {
            "Datum": corrected_date.isoformat(),
            "Wochentag": corrected_date.strftime("%A"),  # durch locale auf Deutsch
            "Dienst": service,
            "Person": msg["person"],
            "Bemerkungen": msg["text"],
            "# Eier": laid_count,
            "# Eier Mitglieder": members_count,
        }
    else:
        # Mehrere Nachrichten zu einem Dienst → Bemerkungen zusammenführen
        records[key]["Bemerkungen"] += " | " + msg["person"] + ": " + msg["text"]
        records[key]["# Eier"] += laid_count
        records[key]["# Eier Mitglieder"] += members_count

# 5) DataFrame erstellen und CSV speichern
df = pd.DataFrame(records.values())
if not df.empty:
    df = df.sort_values(["Datum", "Dienst"], ascending=[True, False])

OUTPUT_PATH = "chat_parsed.csv"
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")
